# NHGF Tools and applications demo: Interpolating NLCD data to Delaware HUC12s

### Introduction

In this example, the 2019 Land Cover product from the National Land Cover Database is interpolated to the Delaware River Basin HUC12s using the Geo Data Processing Tools (gdptools) grid-to-polygon area-weighted statistics tools.

### Geo Data Processing Tools (gdptools)

gdptools is a python package for calculating area-weighted statistics of grid-to-polygon. grid-to-line, and polygon-to-polygon interpolations. gdptools utilizes metadata from the OPeNDAP catalog to aid generalizing gdptools processing capabilities. There are also methods for calculating area-weighted statistics for any gridded dataset that can be opened by xarray, however some of the metadata encapsulated in the json entries will have to be added by the user. Examples can be found in Non-Catalog Examples.

### Open Source Python Tools

- [xarray](http://xarray.pydata.org/en/stable/index.html): Package to work with labelled multi-dimensional arrays.
- [geopandas](http://xarray.pydata.org/en/stable/index.html): Package to work with geospatial data.
- [HyRiver](https://docs.hyriver.io/): Hydroclimate Data Retriever, suite of libraries for retrieving geospatial data from various web services. pynhd library used to retrieve NHDplus water basin polygons and data from the Hydro Network-Linked Data Index (NLDI).
- [Holoviz](https://holoviz.org/): Python data visualization suite. Hvplot used to plot geospatial data.

### Data
- [2019 NLCD Land Cover](https://www.mrlc.gov/): 2019 conterminous U.S. land cover at a 30-meter spatial resolution with a 16-class legend based on a modified Anderson Level II classification system.

### Maintenance Schedule
- The intent of this notebook is simply a demonstration of an application or workflow. As we progress in the build-out of NGHF project tools, this notebook may be updated or replaced. There is no other planned maintenance at this time.

### Description

This notebook demonstrates usage of gdptools to interpolate NLCD 30m land cover data to HUC12s for the Delaware River Basin. There are three sections:
1) Data exploration and pre-processing
2) Calculating zonal statistics for gridded land cover on HUC12 watershed basins
3) Visualization of results

### Requirements
- Linux OS or Windows OS
- Conda-based package manager such as miniconda or mamba

### Create conda environment
- Download [environment-examples.yml](https://code.usgs.gov/wma/nhgf/toolsteam/gdptools/-/blob/develop/environment-examples.yml)
- To create the conda environment, follow the steps below.

```
conda env create -f environment-examples.yml
conda activate gdptools-examples
jupyter lab
```

# 1) Data Exploration and data preparation
### Import Libraries

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely import box
import cartopy
import hvplot.pandas
import hvplot.xarray
import numpy as np
import warnings
import rasterio as rio
import xarray as xr
import rioxarray as rxr
from pynhd import NLDI
from pynhd import WaterData
import pygeohydro as gh
from pygeohydro import helpers

import hvplot.xarray
import hvplot.pandas

from gdptools import WeightGen
from gdptools import AggGen
from gdptools import UserTiffData
from gdptools import UserCatData

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

warnings.filterwarnings("ignore")

### Create GeoDataFrame of Delaware River Basin HUC12s



In [2]:
# USGS gage 01482100 Delaware River at Del Mem Bridge at Wilmington De
gage_id = "01482100"
nldi = NLDI()
del_basins = nldi.get_basins(gage_id)
huc12_basins = WaterData("huc12").bygeom(del_basins.geometry[0])
huc12_basins = huc12_basins[huc12_basins.huc12.str.match('020401')]
shp_crs = huc12_basins.crs.to_wkt()
# huc12_basins = huc12_basins.head(n=200)
len(huc12_basins)

ServiceError: Service returned the following error message:
URL: https://labs.waterdata.usgs.gov/geoserver/wmadata/ows
ERROR: <?xml version="1.0" encoding="UTF-8"?><ows:ExceptionReport xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="2.0.0" xsi:schemaLocation="http://www.opengis.net/ows/1.1 http://labs.waterdata.usgs.gov/geoserver/schemas/ows/1.1.0/owsAll.xsd">
<ows:Exception exceptionCode="NoApplicableCode">
<ows:ExceptionText>Could not parse CQL filter list.
Encountered &amp;quot;Infinity&amp;quot; at line 1, column 32.
Was expecting one of:
    &amp;quot;-&amp;quot; ...
    &amp;lt;INTEGER_LITERAL&amp;gt; ...
    &amp;lt;FLOATING_LITERAL&amp;gt; ...
     Parsing : INTERSECTS(the_geom, POLYGON ((Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, Infinity Infinity, In

In [ ]:
huc12_basins.head()

Create a plot of the HUC12 basins.

In [ ]:
drb = huc12_basins.hvplot(
    geo=True, coastline='50m', alpha=0.2, c='r', frame_width=300,
    xlabel="longitude", ylabel="latitude", clabel="monthly water evaporation (mm)",
    title="Delaware River HUC12 basins", xlim=(-76.8, -73.8), aspect='equal',
    tiles='EsriTerrain',
)

drb

### Pull NLCD raster data using ClimateR Catalog

The following dataset assets are available in the ClimateR catalog from the National Land Cover Database (NLCD) for the Contiguous United States (L48):
- Land Cover
- Tree Canopy
- Impervious Surface

L48 Land Cover is available for 2001, 2004, 2006, 2008, 2011, 2016, and 2019 in the ClimateR catalog, with more limited temporal coverage for other variables at the time of writing in 2023, and more limited variables and years for Alaska (AK), Hawaii (HI), and Puerto Rico (PR).

Pandas is used to query the catalog based on the asset of interest, and the json metadata is returned for the asset.

In [ ]:
#%% 
climater_cat = "https://mikejohnson51.github.io/climateR-catalogs/catalog.parquet"
cat = pd.read_parquet(climater_cat)
cat.head()

### Query catalog for NLCD 2019 Land Cover L48

In [ ]:
_id = "NLCD"
_asset = "2019 Land Cover L48"

# an example query returns a pandas dataframe.
pd.set_option('display.max_colwidth', 100)
tc = cat.query("id == @_id & asset == @_asset")
tc

### Load queried NLCD catalog asset into Rioxarray

In [ ]:
ds = rxr.open_rasterio(tc.URL.values[0])
data_crs = ds.spatial_ref.crs_wkt
ds

### Set up input data
For climater-catalog entries that are of type `vrt` or for any rasters, gdptools provides the [`UserTiffData`](https://gdptools.readthedocs.io/en/develop/user_input_data_classes.html#module-gdptools.data.user_data) class.  The `UserTiffData` class requires the user to define parameters for both the source and target data.  Here the source data parameters are:
- var: Not currently used so a dummy variable can be passed
- ds: the xarray dataset
- proj_ds: projection for the xarray dataset which can be retrieved from the crs_wkt attribute of the spatial_ref var, or it can be retrieved from the crs attribute of the climater-catalog query
- x_coord, y_coord: Coordinate names determined from the xarray dataset
- band, bname: Both determined from the xarray dataset

The target data parameters are:
- f_feature: The Geopandas dataframe
- id_feature: The string of the id used to associate the results of the zonal stats
- proj_feature: The projection or crs of the source data.

In [ ]:
x_coord = 'x'
y_coord = 'y'
band='band'
var = 'descriptor_2019'
shp_poly_idx = 'huc12'

user_data = UserTiffData(
    var=var,
    ds=ds,
    proj_ds=data_crs,
    x_coord=x_coord,
    y_coord=y_coord,
    band=1,
    bname=band,
    f_feature=huc12_basins,
    proj_feature=shp_crs,
    id_feature=shp_poly_idx,
)

user_data

# 2) Calculating zonal statistics for gridded land cover on HUC12 watershed basins
### Calculate zonal majority land cover classes
ZonalGen() class is the object to calculate the zonal mean using the calculate_zonal() method.  

In [ ]:
from gdptools import ZonalGen

zonal_gen = ZonalGen(
    user_data=user_data,
    zonal_engine="serial",
    zonal_writer="csv",
    out_path=".",
    file_prefix="Delaware_NLCD_stats"
)

stats = zonal_gen.calculate_zonal(categorical=True)

In [ ]:
print(stats.head(n=20))

### Add majority land cover category to geodataframe

In [ ]:
huc12_basins.sort_values(by=shp_poly_idx, inplace=True)
stats.sort_values(by=shp_poly_idx, inplace=True)
huc12_basins["TEXT"] = stats["top"].values
huc12_basins.head()


# 3) Visualization of results
### Import NLCD land cover lookup information

In [ ]:
nlcd_lookup = helpers.nlcd_helper()['classes']
del nlcd_lookup['127']
del nlcd_lookup['51']
del nlcd_lookup['72']
del nlcd_lookup['73']
del nlcd_lookup['74']
print(len(nlcd_lookup))
nlcd_lookup

### Set colors for land cover classes
hex codes follow color scheme from [us-land-cover](https://github.com/BuzzFeedNews/us-land-cover).

In [ ]:
len(nlcd_lookup)
nlcd_colors = ['#5475a8', '#ffffff', '#e8d1d1', '#e29e8c', '#ff0000', '#b50000', '#d2cdc0', '#85c77e', '#38814e', '#d4e7b0', '#dcca8f', '#d4e7b0', '#dcca8f', '#e2e2c1', '#fbf65d', '#ca9146', '#c8e6f8', '#64b3d5']
len(nlcd_colors)

### Create dataframe from NLCD lookup dictionary

In [ ]:
nlcd_lookup_df = pd.DataFrame.from_dict({'code': [int(i) for i in list(nlcd_lookup.keys())], 'category': list(nlcd_lookup.values()), 'colors': nlcd_colors})
nlcd_lookup_df['land cover'] = nlcd_lookup_df['category'].apply(lambda name: name.split(' -')[0])
nlcd_lookup_df

### Create categorical colormap from lookup dataframe

In [ ]:
color_lookup = {}
for i in range(len(nlcd_lookup_df)):
    key = nlcd_lookup_df['land cover'][i]
    color_lookup[key]=nlcd_lookup_df['colors'][i]

print(color_lookup)

### Add land cover lookup information to geodataframe

In [ ]:
huc12_basins = huc12_basins.merge(nlcd_lookup_df, left_on='TEXT', right_on='code', how='inner', copy=True)
huc12_basins['TEXT'] = huc12_basins['TEXT'].apply(str)
huc12_basins.head()

### Plot zonal majority land cover classes

In [ ]:
huc12_basins.hvplot(
    geo=True, 
    c='land cover', 
    cmap=color_lookup,
    coastline='50m',
    alpha=0.8, 
    frame_width=300,
    xlabel="longitude", 
    ylabel="latitude", 
    title="Test HUC12 basin ", 
    aspect='equal',
    tiles='OSM',
)

### Validation
- Clip land cover raster to a single HUC12 (first in the huc12_basins geodataframe) and reproject data
- Determine the faction of each class in the clipped raster
- Plot the resulting clipped raster and HUC12

#### Clip Raster

In [ ]:
gdf_bounds = huc12_basins.head(n=1).to_crs(data_crs).total_bounds
minx, miny, maxx, maxy = gdf_bounds

clipped = ds.rio.clip_box(minx, miny, maxx, maxy)

clipped_reprojected = clipped.rio.reproject("EPSG:4326")
clipped_reprojected

#### Use categorical describe() method to determine top value in clipped raster

In [ ]:
filtered_array = clipped_reprojected.values[0,:,:]
filtered_array = filtered_array[filtered_array != 255]
pd.DataFrame(filtered_array.ravel(), dtype="category").describe()
# pd.DataFrame(clipped_reprojected.values[0,:,:].ravel(), dtype="category").describe()

#### Determine the fraction of each category in the clipped raster
Above, we see that the most common category is 41, Deciduous Forest. Clipping the land cover raster to the bounds of the HUC12 watershed polygon leaves in grid cells that fall outside the polygon, but we can now start to understand why 41 is the top land cover class in the zonal statistics results for this HUC12 polygon. Below, each of the land cover classes are listed along with the corresponding proportion of cells in the subset made up of that land cover class. We see that nearly half of the cells, 45.0%, are Deciduous Forest, while the next most-abundant land cover class is 81, Pasture/Hay, at 26.9%.

In [ ]:
# Getting unique values and their counts
unique_values, counts = np.unique(filtered_array, return_counts=True)

# Calculating frequencies
frequencies = counts / filtered_array.size

list(zip(unique_values, frequencies))

### Plot single HUC12 and clipped gridded land cover data

In [ ]:
newds = clipped_reprojected.to_dataset(name="nlcd").isel(band=0)
newds

In the figure below, note the prevalence of the medium green corresponding to code 41, Deciduous Forest, and yellow, corresponding to 81, Pasture/Hay.

In [ ]:
import matplotlib.pyplot as plt
cmap, norm, levels = gh.plot.cover_legends()
fig, ax = plt.subplots(figsize=(10,8))
newds['nlcd'].plot(ax=ax, cmap=cmap, levels=levels, cbar_kwargs={"ticks": levels[:-1]})
huc12_basins.head(n=1).plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

### Summary
In this notebook we have shown how to access NLCD data through the ClimateR-Catalog.  In addition we've shown how to generate zonal statistics using GDPTools.  The results of the top or most prevalent value in each polygon was added to the GeoDataFrame and plotted.  As a simple verification, we plotted the raw data and determined the fraction of each landcover type.  The dominant type in the clipped raster associated with the first geometry in the GeoDataFrame conincides with the zonal stats value for "top" or most prevalent value.  

Note: we used the ClimateR-Catalog to provide a link to NLCD data, The complete NLCD data are not available in the Catalog.  One could also download the NLCD data of interest and run a similar analysis.  An Example of calculating raster statistics using GDPTools can be found [here](https://gdptools.readthedocs.io/en/develop/Examples/Rasters/zonal_stats.html)